In [2]:
import math

import numpy as np
from rdp import rdp
import time
from matplotlib import pyplot as plt
import pandas as pd
from os.path import join
import os
import math
from utilities import runQueryDTheta, loadData, donor_acceptor_dist, spm
# %matplotlib widget

dataPath = r"C:\Users\ashamail\Box\Shared Materials_Abdullah_Hasan\Abdullah Data\allAtomsWithRings.csv"
savePath = r"C:\Users\ashamail\Desktop\drug-polymers\adbis extension\data\ellipsoids"
savepathDrive = r"C:\Users\ashamail\OneDrive - Iowa State University\Molecular Simulation Data\adbis extension data"
oxygenPath = r"C:\Users\ashamail\OneDrive - Iowa State University\Molecular Simulation Data\Text Data Files\flavanone_R1_os_0-200K.csv"
oxygen_location = r"C:\Users\ashamail\Desktop\data\os_200_1.txt"
nitrogenPath = r"C:\Users\ashamail\Desktop\data\nitrogen200k.txt"



In [ ]:
def performRDP(data, epsilon):
    rdpData = rdp(data, epsilon=epsilon, return_mask=True)
    myData = []
    for i, truth in enumerate(rdpData):
        if truth:
            temp = list(np.append(data[i,:], i))
            myData.append(temp)
    myData = np.array(myData)
    return myData

In [3]:
data = pd.read_csv(dataPath)
# oxygen = pd.read_csv(oxygenPath, header=None)
# oxygen = oxygen.rename(columns={0: "x", 1: "y", 2: "z"})
oxygen = loadData(oxygen_location)
nitrogen = loadData(nitrogenPath)
min_z_points = pd.read_csv(join(savepathDrive, "min_z_points.csv"))

PSP = 44

In [5]:
methods = ["spm", "rdp"]
saveLocation = r"C:\Users\ashamail\OneDrive - Iowa State University\Molecular Simulation Data\adbis extension data\2_spm_rdp"

epsilons = [0.5, 0.75, 1]


toappend = np.arange(200000)
toappend = toappend.reshape(toappend.shape[0], 1)
rflOxygen = np.append(oxygen[:,0,:], toappend, axis = 1)


for method in methods:
    for epsilon in epsilons:
        s1 = time.time()
        if method == "spm":
            out = spm(min_z_points[["min_z", "timestep"]].values, epsilon)
        else:
            out = performRDP(min_z_points[["min_z", "timestep"]].values, epsilon)
        timesteps = out[:, 2].astype(int)
        
        print(method, epsilon, "pre-processing", time.time() - s1)

        print(method, epsilon, "done with pre processing, starting file saving")

        temp = rflOxygen[timesteps]
        np.savetxt(join(saveLocation, method, str(epsilon), "oxygen.csv"), temp, delimiter=",")
        # tempnitrogen = nitrogen[timesteps]
        # for i in range(216):
        # #     nitrogen[:,i,:].shape
        #     nitrogeni = np.append(tempnitrogen[:,i,:], timesteps.reshape(timesteps.shape[0], 1), axis = 1)
        #     np.savetxt(join(saveLocation, method, str(epsilon), str(i)+"-nitrogen.csv"), nitrogeni, delimiter=",")
        print(method, epsilon, "done with saving files, starting query")
        s = time.time()
        a, b = donor_acceptor_dist(oxygen[:,0,:], nitrogen, timesteps, hbond_threshold = 3.5 + 2*epsilon)
        print(method, epsilon, "query processing", a.shape, time.time() - s)

spm 0.5 pre-processing 46.94548678398132
spm 0.5 done with pre processing, starting file saving
spm 0.5 done with saving files, starting query
spm 0.5 query processing (725,) 25.12169575691223
spm 0.75 pre-processing 20.467148303985596
spm 0.75 done with pre processing, starting file saving
spm 0.75 done with saving files, starting query
spm 0.75 query processing (1091,) 17.10965871810913
spm 1 pre-processing 10.86345362663269
spm 1 done with pre processing, starting file saving
spm 1 done with saving files, starting query
spm 1 query processing (1201,) 11.255786418914795
rdp 0.5 pre-processing 212.91887426376343
rdp 0.5 done with pre processing, starting file saving
rdp 0.5 done with saving files, starting query
rdp 0.5 query processing (772,) 24.85229182243347
rdp 0.75 pre-processing 202.31624507904053
rdp 0.75 done with pre processing, starting file saving
rdp 0.75 done with saving files, starting query
rdp 0.75 query processing (1287,) 16.69910764694214
rdp 1 pre-processing 197.901

In [6]:
# methods = ["rdp"]
# saveLocation = r"C:\Users\ashamail\OneDrive - Iowa State University\Molecular Simulation Data\adbis extension data\2_spm_rdp"

# epsilons = [0.5, 0.75, 1]


# toappend = np.arange(200000)
# toappend = toappend.reshape(toappend.shape[0], 1)
# rflOxygen = np.append(oxygen[:,0,:], toappend, axis = 1)


# for method in methods:
#     for epsilon in epsilons:
#         s1 = time.time()
#         if method == "spm":
#             out = spm(min_z_points[["min_z", "timestep"]].values, epsilon)
#         else:
#             out = performRDP(min_z_points[["min_z", "timestep"]].values, epsilon)
#         timesteps = out[:, 2].astype(int)
        
#         print(method, epsilon, "pre-processing", time.time() - s1)

#         print(method, epsilon, "done with pre processing, starting file saving")

#         # temp = rflOxygen[timesteps]
#         # np.savetxt(join(saveLocation, method, str(epsilon), "oxygen.csv"), temp, delimiter=",")
#         # tempnitrogen = nitrogen[timesteps]
#         # for i in range(216):
#         # #     nitrogen[:,i,:].shape
#         #     nitrogeni = np.append(tempnitrogen[:,i,:], timesteps.reshape(timesteps.shape[0], 1), axis = 1)
#         #     np.savetxt(join(saveLocation, method, str(epsilon), str(i)+"-nitrogen.csv"), nitrogeni, delimiter=",")
#         print(method, epsilon, "done with saving files, starting query")
#         s = time.time()
#         a, b = donor_acceptor_dist(oxygen[:,0,:], nitrogen, timesteps, hbond_threshold = 3.5 + 2*epsilon)
#         print(method, epsilon, "query processing", a.shape, time.time() - s)